In [ ]:
#| notest
#| eval: false
#| hide

import sys

sys.path.insert(0, '../unreal_llm_sandbox/')
sys.path.insert(0, '../../../')

import big_project_helper as bph

bph.display_project_contents('unreal_llm_sandbox')

Dialog Name: unreal/unreal-llm-sandbox/nbs/main

../unreal_llm_sandbox/llm_tools.py
# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/llm_tools.ipynb.

# %% auto 0
__all__ = ['TOOLS', 'TOOL_SCHEMAS', 'get_tools', 'register_tool', 'search_web', 'read_url']

# %% ../nbs/llm_tools.ipynb 2
import requests
import litellm
import json
from lisette import lite_mk_func

TOOLS = {}
TOOL_SCHEMAS = []


def get_tools():
    """Return available tool schemas."""
    return TOOL_SCHEMAS


def register_tool(func):
    """Register a function as a tool."""
    TOOLS[func.__name__] = func
    schema = lite_mk_func(func)
    # Remove old schema if exists
    TOOL_SCHEMAS[:] = [s for s in TOOL_SCHEMAS if s['function']['name'] != func.__name__]
    TOOL_SCHEMAS.append(schema)
    return func


@register_tool
def search_web(query: str, max_results: int = 10):
    """Search DuckDuckGo and return top results.
    
    Args:
        query: Search string.
        max_results: Maximum number of results to return

In [ ]:
#| default_exp main


## Imports & FastHTML App Setup
- Imports core modules: json, asyncio, requests, lisette, fasthtml
- Imports local modules: config, scripts, cells, streaming, kernel, notebook_io, llm, agent
- Sets up DaisyUI headers with Tailwind, Prism, Monaco editor, and marked.js
- Initializes FastHTML app with JupyUvi server

*Auto-Summary*

In [ ]:
#| export
import json
import asyncio
import requests
import lisette
import time

from fasthtml.common import *
from fasthtml.jupyter import JupyUvi
from starlette.staticfiles import StaticFiles

from unreal_llm_sandbox.app_config import MODEL, KERNEL_URL, NOTEBOOK_SYS_PROMPT
from unreal_llm_sandbox.cells import MarkdownCell, CodeCell, PromptCell, AgentCell
from unreal_llm_sandbox.streaming import SSEStream, active_streams
from unreal_llm_sandbox.kernel import execute_unreal_code, convert_to_accumulated
from unreal_llm_sandbox.notebook_io import reconstruct_cells_from_history, prepare_chat_history, reconstruct_ipynb_cell
from unreal_llm_sandbox.llm import RemoteToolLLM, send_llm_request
from unreal_llm_sandbox.agent import AgentTools, SYS_PROMPT
from unreal_llm_sandbox.scripts import CODE_CELL_SCRIPT, HEIGHT_ADJ_SCRIPT, CELL_FOCUS_SCRIPT

from fasthtml.common import *
import importlib.resources
import unreal_llm_sandbox

# 1. Helper function to read the text content of your files
def get_static(fname):
    # This looks inside: unreal_llm_sandbox/static/
    ref = importlib.resources.files(unreal_llm_sandbox) / 'static' / fname
    return ref.read_text()


daisy_hdrs =[
Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism-okaidia.min.css"),
Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/github-markdown-css/5.5.1/github-markdown.min.css"),
Script (src ='https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js'),
Script (src ='https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-python.min.js'),
Script(src='https://cdn.jsdelivr.net/npm/marked/marked.min.js'),
Script(get_static('cells.js')),
Script(src="https://cdn.jsdelivr.net/npm/ansi_up@5/ansi_up.min.js"),
Script(src="https://cdn.jsdelivr.net/npm/monaco-editor@latest/min/vs/loader.js"),
Script("""
require.config({ paths: { 'vs': 'https://cdn.jsdelivr.net/npm/monaco-editor@latest/min/vs' }});
"""),
Script(src="https://unpkg.com/htmx.org/dist/ext/sse.js")]


# FastHTML app setup + daisy_hdrs (the big Style/Script list)
app = FastHTML(hdrs=daisy_hdrs)
#app.mount("/static", StaticFiles(directory="static"), name="static")

rt = app.route


## Stream Interrupt Route
- `POST /interrupt/{cell_id}` - Signals abort for active SSE streams
- Uses notebook:cell_id composite key for stream identification

*Auto-Summary*

In [ ]:
#| export

@rt('/interrupt/{cell_id}', methods=['POST'])
async def interrupt(cell_id: str, request):
    """Signal abort for an active stream.
    
    Args:
        cell_id: Unique cell identifier.
    
    Returns:
        "OK" acknowledgment string.
    """
    try:
        data = await request.json()
        notebook = data.get('notebook', 'untitled')
    except:
        notebook = 'untitled'
    stream_key = f"{notebook}:{cell_id}"
    if stream_key in active_streams:
        active_streams[stream_key]['abort'] = True
    return "OK"
    

## LLM Prompt Execution Route
- `GET /execute_prompt/{cell_id}` - Executes LLM prompts with notebook context
- Converts cell history to chat format and streams response via SSE

*Auto-Summary*

In [ ]:
#| export

@rt('/execute_prompt/{cell_id}')
async def exe_prompt(cell_id: str, request): 
    """Execute LLM prompt with notebook context via SSE.
    
    Args:
        cell_id: Unique cell identifier.
        request: FastHTML request with JSON body containing:
            - prompt: User's prompt text.
            - context: List of cell dicts for history.
    
    Returns:
        SSE StreamingResponse yielding LLM chunks.
    """

    data = await request.json() 

    notebook = data.get('notebook', 'untitled')
    stream_key = f"{notebook}:{cell_id}" 

    prompt = data['prompt']
    cell_dict_list = data.get('context', [])
    
    cells = reconstruct_cells_from_history(cell_dict_list)
    ipynb_list = [cell.to_ipynb() for cell in cells]
    chat_history = prepare_chat_history(ipynb_list)
    
    stream = SSEStream(stream_key)
    
    def run():
        for msg in send_llm_request(prompt, history=chat_history):
            if stream.aborted(): break
            stream.text(msg)
        stream.done()
    
    asyncio.create_task(asyncio.to_thread(run))
    return stream.response()


## Code Execution Route
- `GET /execute_code/{cell_id}` - Executes Python code in Unreal Engine kernel
- Streams kernel output messages via SSE

*Auto-Summary*

In [ ]:
#| export

@rt('/execute_code/{cell_id}')
async def exe_code(cell_id: str, request):
    """Execute Python code in Unreal Engine via SSE.
    
    Args:
        cell_id: Unique cell identifier.
        request: FastHTML request with JSON body containing:
            - code: Python code string to execute.
    
    Returns:
        SSE StreamingResponse yielding kernel output messages.
    """

    data = await request.json()
    notebook = data.get('notebook', 'untitled')
    stream_key = f"{notebook}:{cell_id}" 

    stream = SSEStream(stream_key)
    
    def run():
        response = requests.post(f'{KERNEL_URL}/execute', json={'code': data['code']}, stream=True, timeout=(5, 60))
        for line in response.iter_lines():
            if stream.aborted(): break
            if line.startswith(b'data: '):
                stream.output(json.loads(line[6:]))  # ← output() not raw()
        stream.done()
    
    asyncio.create_task(asyncio.to_thread(run))
    return stream.response()


## Agent Tool Builder Route
- `POST /agent_tool_build/{cell_id}` - Runs agentic code generation loop
- Supports tool calling, code improvement iterations, and unit testing
- Streams progress (tags, text, outputs) via SSE

*Auto-Summary*

In [ ]:
#| export

@rt('/agent_tool_build/{cell_id}', methods=['POST'])
async def agent_stream(cell_id: str, request):
    """Run agent code generation loop with tool calling via SSE.
    
    Args:
        cell_id: Unique cell identifier.
        request: FastHTML request with JSON body containing:
            - prompt: Code generation request.
            - existing_code: Optional code to modify.
            - context: List of cell dicts for history.
    
    Returns:
        SSE StreamingResponse yielding agent progress (tags, text, outputs).
    """
    data = await request.json()
    notebook = data.get('notebook', 'untitled')
    stream_key = f"{notebook}:{cell_id}" 

    stream = SSEStream(stream_key)
    def run_chat():
        PROMPT = data['prompt']

        existing_code = data.get('existing_code')
        cell_dict_list = data.get('context', [])  # ← Add this

        # Convert to chat history like prompt cells do
        cells = reconstruct_cells_from_history(cell_dict_list)
        ipynb_list = [cell.to_ipynb() for cell in cells]
        chat_history = prepare_chat_history(ipynb_list)

        if existing_code:
            CODE = existing_code  # Initialize CODE with existing
            PROMPT = f"Modify this code: {existing_code}\n\nRequest: {data['prompt']}"
        else:
            CODE = ""
            PROMPT = data['prompt']
        

        CHAT = lisette.Chat(MODEL, SYS_PROMPT)
        CHAT.hist += chat_history
        CHAT.hist.append( {"role":"assistant", "content":PROMPT})

        a_tools = AgentTools(stream, CHAT, PROMPT,cell_id, code=CODE, print_updates=False)
        tools = a_tools.get_tools()
        
        chat = lisette.Chat(MODEL, SYS_PROMPT, tools=tools)
        chat.hist += chat_history


        gen = chat(PROMPT, max_steps=15)
        for _ in gen:
            if stream.aborted():
                break
        print("Chat loop finished!") 
        time.sleep(0.1) 
        stream.done()
    
    asyncio.create_task(asyncio.to_thread(run_chat))
    return stream.response()
    

## Styling & Toolbar UI
- Defines dark theme CSS styles for markdown body and content areas
- `Toolbar()` component with notebook name input and cell type buttons
- `GET /add_cell/{cell_type}` - Creates new cells (markdown, code, prompt, agent)

*Auto-Summary*

In [ ]:
#| export

def Toolbar(title):
    return Div(

        Div(
            Input(value=title, cls="text-xl font-bold text-white notebook-name bg-transparent border-none outline-none focus:outline-none flex-1"),
            Script("""
                document.querySelector('.notebook-name').addEventListener('blur', (e) => {
                    const name = e.target.value || 'untitled';
                    history.replaceState(null, '', `/notebook/${name}.ipynb`);
                });
            """),
            cls="flex flex-1 items-center"
        ),
        Div(
            Button("➕ Markdown", 
                   hx_post="/add_cell/markdown",
                   hx_target="#notebook-container",
                   hx_swap="beforeend",
                   cls="btn btn-sm bg-gray-700 hover:bg-gray-600 text-white border-gray-600 shadow-none"), 
            Button("➕ Code", 
                   hx_post="/add_cell/code",
                   hx_target="#notebook-container",
                   hx_swap="beforeend",
                   cls="btn btn-sm bg-gray-700 hover:bg-gray-600 text-white border-gray-600 shadow-none"),  
            Button("➕ Prompt", 
                   hx_post="/add_cell/prompt",
                   hx_target="#notebook-container",
                   hx_swap="beforeend",
                   cls="btn btn-sm bg-gray-700 hover:bg-gray-600 text-white border-gray-600 shadow-none"),  
            Button("➕ Agent", 
                   hx_post="/add_cell/agent",
                   hx_target="#notebook-container",
                   hx_swap="beforeend",
                   cls="btn btn-sm bg-gray-700 hover:bg-gray-600 text-white border-gray-600 shadow-none"), 
            cls="flex gap-2"
        ),
        cls="flex justify-between p-4 bg-[#0d0d0d]"  # ← Changed from #0d0d0d to pure black
    )

@rt('/add_cell/{cell_type}')
def add_cell(cell_type: str):
    """Create and return a new cell of the specified type.
    
    Args:
        cell_type: One of 'markdown', 'code', 'prompt', or 'agent'.
        
    Returns:
        Ren
    """
    if cell_type == 'markdown':
        new_cell = MarkdownCell("")
    elif cell_type == 'code':
        new_cell = CodeCell("")
    elif cell_type == 'prompt':
        new_cell = PromptCell("")
    elif cell_type == 'agent':
        new_cell = AgentCell("")
    else:
        return "Invalid cell type"
    
    return new_cell.render()


## Notebook Load & Save Routes
- `GET /notebook/{notebook_file}` - Loads .ipynb file and renders cells with toolbar
- `POST /save_notebook/{notebook_file}` - Saves cells to Jupyter notebook format

*Auto-Summary*

In [ ]:
#| export

@rt('/notebook/{notebook_file}')
def load_notebook(notebook_file:str): 
    """Load a Jupyter Notebook file and render its cells."""

    if not os.path.exists(notebook_file):
        rendered_cells = []
        print ('Notebook Not Found:',notebook_file)

    else:
        cells = []
        with open(notebook_file, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
            cells = notebook['cells']
        
        cell_objects = [reconstruct_ipynb_cell(cell) for cell in cells]
        rendered_cells = [cell.render() for cell in cell_objects]


    return Title("Unreal LLM Sandbox"),Body(
        Toolbar(notebook_file.split('.ipynb')[0]),
        Style(get_static('styles.css')),
        Div(  *rendered_cells,
            cls='px-5',  
            id='notebook-container' 
        )
    )

@rt('/save_notebook/{notebook_file}', methods=['POST'])
async def save_notebook(notebook_file: str, request):
    """Save notebook cells to a Jupyter .ipynb file.
    
    Args:
        notebook_file: Filename to save to.
        request: Request with JSON body containing 'cells' list.
    
    Returns:
        JSON with status message.
    """
    data = await request.json()
    cell_dict_list = data.get('cells', [])
    
    # Reconstruct cell objects and convert to ipynb format
    cells = reconstruct_cells_from_history(cell_dict_list)
    ipynb_cells = [cell.to_ipynb() for cell in cells]
    
    # Build notebook structure
    notebook = {
        "nbformat": 4,
        "nbformat_minor": 5,
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            }
        },
        "cells": ipynb_cells
    }
    
    with open(notebook_file, 'w', encoding='utf-8') as f:
        json.dump(notebook, f, indent=2, ensure_ascii=False)
    
    return {"status": "saved", "file": notebook_file}
    

In [ ]:
#| export
import uvicorn

def start_server():
    uvicorn.run(app, host='0.0.0.0', port=5001)
    

In [ ]:
#| export
#| eval: false
if __name__ == "__main__":
    start_server()

RuntimeError: asyncio.run() cannot be called from a running event loop

# For Notebook Testing

In [ ]:
#| eval: false
from fasthtml.jupyter import JupyUvi
server = JupyUvi(app)

# Reflect
- How does this relate to the larger project? List the other components and how they communicate with this one. 
- What improvements do you see that can obviously be made here? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

## How This Relates to the Larger Project

This is a **Jupyter-like notebook web app** that connects to an **Unreal Engine 5.6** Python kernel. The components:

| Component | Role | Communication |
|-----------|------|---------------|
| `main.py` (FastHTML routes) | Web server, SSE streaming | HTTP/SSE to browser, HTTP to Unreal kernel |
| `cells.py` | Cell rendering (Markdown, Code, Prompt, Agent) | Generates HTML for frontend |
| `kernel.py` | Unreal Engine bridge | HTTP POST to `KERNEL_URL` for code execution |
| `agent.py` | Agentic code generation loop | Uses `lisette` for tool-calling LLM |
| `llm.py` | LLM chat with tool execution | LiteLLM + remote tool calls to Unreal |
| `streaming.py` | SSE queue management | Async queue → StreamingResponse |
| `notebook_io.py` | ipynb ↔ cell conversion | JSON parsing/serialization |
| `scripts.py` | Frontend JavaScript | Handles Monaco, cell execution, SSE consumption |

## Obvious Improvements

1. **Error handling** - Many `try/except` blocks swallow errors silently or use bare `except:`
2. **Hardcoded `MODEL = 'gpt-5.2'`** - Should be configurable/validated
3. **No authentication** - Anyone can execute code on the Unreal kernel
4. **Duplicate code** - `RemoteToolLLM` and `AgentTools` both implement tool execution
5. **Magic strings** - Separators like `PROMPT_SPLIT` are fragile; could use structured metadata
6. **JavaScript in Python strings** - `scripts.py` is hard to maintain; could be separate `.js` files
7. **No rate limiting** on kernel execution or LLM calls

Which area would you like to explore further?

# Consolidating CSS

In [ ]:
from pathlib import Path

Path("static").mkdir(exist_ok=True)

css = """
.markdown-body { 
    background-color: bg-gray-900 !important; 
}
.markdown-body ul { 
    list-style-type: disc !important; 
    padding-left: 2em !important; 
}
.markdown-body ol { 
    list-style-type: decimal !important; 
    padding-left: 2em !important; 
}
[data-cell-id].minimized .output-display-llm,
[data-cell-id].minimized .output-display-code {
    display: none !important;
}
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render,
[data-cell-id].minimized .output-display,
[data-cell-id].minimized .text-gray-400 {
    display: none;
}
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render {
    display: none !important;
}
[data-cell-id].minimized .monaco-editor,
[data-cell-id].minimized .content-edit,
[data-cell-id].minimized .content-render,
[data-cell-id].minimized .output-display {
    max-height: 0 !important;
    overflow: hidden;
}
[data-cell-id].minimized .bg-gray-800 {
    opacity: 0.5 !important;
}
body {
    background-color: #070707 !important;  /* 95% black = 5% brightness */
}
.markdown-body { 
    background-color: bg-gray-900 !important;
    font-size: 0.825rem !important;  /* ← Smaller base size */
}
.markdown-body h1 { font-size: 1.25rem !important; }
.markdown-body h2 { font-size: 1.1rem !important; }
.markdown-body h3 { font-size: 1.0rem !important; }
.markdown-body ul, .markdown-body ol { 
    list-style-type: disc !important; 
    padding-left: 2em !important;
    font-size: 0.825rem !important;  /* ← Match body */
}
.content-edit {
    font-size: 0.825rem !important;  /* ← Textarea size */
}

.content-render {
    background-color: #1a1a1a  !important;  /* ← Medium grey for rendered input */
}
.running { color: #ef4444 !important; }
.complete { color: #22c55e !important; }
#notebook-container {
    height: calc(100vh - 64px); 
    overflow-y: auto;
.toggle-edit:checked ~ .content-edit { display: block; }
.toggle-edit:checked ~ .content-render { display: none; }
.toggle-edit:not(:checked) ~ .content-edit { display: none; }
.toggle-edit:not(:checked) ~ .content-render { display: block; }
"""

Path("static/styles.css").write_text(css)

# Consolidating JS